# Integrate a regression tree model to classify a person's position based on landmark distances
- Collect Training Data: Capture images with known position labels (e.g., "Standing", "Push-up Up", "Push-up Down", "Lying Down").
- Extract Features: Compute landmark distances (e.g., shoulder-to-hip, hip-to-knee, elbow angle, etc.).
- Train a Decision Tree: Use scikit-learn's DecisionTreeClassifier or DecisionTreeRegressor to classify or predict positions.
- Integrate into Real-Time Detection: Use the trained model to classify the current pose.

---
# Embeddings in a CSV file

---

In [7]:
import cv2
import mediapipe as mp
import csv
import numpy as np
import os

In [ ]:
# Initialize MediaPipe Pose module
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Directory containing labeled folders with images
base_folder = "./training_datasets/push-ups_images"  # Change to your folder path

# Create CSV file and write headers
output_folder ="./models"
csv_filename = "push-up_landmark_positions.csv"
headers = ["shoulder_hip_dist", "hip_knee_dist", "elbow_angle", "label"]

with open(os.path.join(output_folder,csv_filename), "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(headers)
    
    for label in os.listdir(base_folder):
        label_folder = os.path.join(base_folder, label)
        if not os.path.isdir(label_folder):
            continue
        
        for image_name in os.listdir(label_folder):
            image_path = os.path.join(label_folder, image_name)
            
            image = cv2.imread(image_path)
            if image is None:
                continue
            
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = pose.process(image_rgb)
            
            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark
                
                # Get key landmarks
                shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER]
                hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP]
                knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE]
                elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW]
                wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST]
                
                # Compute distances
                shoulder_hip_dist = np.linalg.norm([shoulder.x - hip.x, shoulder.y - hip.y])
                hip_knee_dist = np.linalg.norm([hip.x - knee.x, hip.y - knee.y])
                
                # Compute elbow angle
                def calculate_angle(a, b, c):
                    ba = np.array([a.x - b.x, a.y - b.y])
                    bc = np.array([c.x - b.x, c.y - b.y])
                    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
                    return np.degrees(np.arccos(np.clip(cosine_angle, -1.0, 1.0)))
                
                elbow_angle = calculate_angle(shoulder, elbow, wrist)
                
                # Save to CSV
                print( [shoulder_hip_dist, hip_knee_dist, elbow_angle, label] )
                writer.writerow([shoulder_hip_dist, hip_knee_dist, elbow_angle, label])
                print(f"Data saved: {shoulder_hip_dist}, {hip_knee_dist}, {elbow_angle}, {label}")
            
            cv2.imshow("Pose Capture", image)
            if cv2.waitKey(500) & 0xFF == ord('q'):
                break
    
cv2.destroyAllWindows()

I0000 00:00:1743506881.011850   27447 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1743506881.074223   27667 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 550.120), renderer: NVIDIA GeForce GTX 1650/PCIe/SSE2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1743506881.175760   27662 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743506881.218348   27664 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1743506881.238337   27663 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


[0.2107882178569644, 0.20439975017300308, 174.22222822224214, 'position_up']
Data saved: 0.2107882178569644, 0.20439975017300308, 174.22222822224214, position_up
[0.1723526185266647, 0.18779656679464757, 174.49778992836016, 'position_up']
Data saved: 0.1723526185266647, 0.18779656679464757, 174.49778992836016, position_up
[0.07676939130570416, 0.0690825706895846, 158.73052503524738, 'position_up']
Data saved: 0.07676939130570416, 0.0690825706895846, 158.73052503524738, position_up
[0.1893029501645426, 0.17377531027653362, 177.1938549376211, 'position_up']
Data saved: 0.1893029501645426, 0.17377531027653362, 177.1938549376211, position_up
[0.22223274703204687, 0.18446803624662683, 174.8141468718528, 'position_up']
Data saved: 0.22223274703204687, 0.18446803624662683, 174.8141468718528, position_up
[0.14129941364604237, 0.01044940657971783, 179.13200462634234, 'position_up']
Data saved: 0.14129941364604237, 0.01044940657971783, 179.13200462634234, position_up
[0.10938023290542945, 0.0703

---
# Classification

---

In [3]:
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import joblib
import os

In [9]:
# Load collected data
data = pd.read_csv(os.path.join(output_folder,csv_filename))  # Columns: distances + labels
# data = data.rename(columns={'pushups_up':'label', 'image_001.jpg':'name'})
# data = data.drop(columns=['name'])
data

,shoulder_hip_dist,hip_knee_dist,elbow_angle,label
0,0.210788,0.204400,174.222228,position_up
1,0.172353,0.187797,174.497790,position_up
2,0.076769,0.069083,158.730525,position_up
3,0.189303,0.173775,177.193855,position_up
4,0.222233,0.184468,174.814147,position_up
5,0.141299,0.010449,179.132005,position_up
6,0.109380,0.070347,178.504524,position_up
7,0.096063,0.106259,177.492142,position_up
8,0.182005,0.133155,179.113701,position_up
9,0.040861,0.073467,134.158057,position_up


In [11]:
X = data.drop(columns=['label'])  # Features: distances
y = data['label']  # Target: position labels

# Train classifier
clf = DecisionTreeClassifier()
clf.fit(X, y)

# Save model
joblib.dump(clf,os.path.join(output_folder,"push-up_classifier.pkl"))

['./models/push-up_classifier.pkl']